<div>
<img src="https://i.ibb.co/v3CvVz9/udd-short.png" width="150"/>
    <br>
    <strong>Universidad del Desarrollo</strong><br>
    <em>Magíster en Data Science</em><br>
    <em>Asignatura Almacenamiento y Captura de Datos</em><br>
    <em>Profesor: Carlos Perez </em><br>

</div>

# **Proyecto: Portal Inmobiliario**

*27 de Diciembre de 2024*

**Nombre Estudiante(s)**: Joaquin Leiva - Victor Saldivia Vera - Cristian Tobar - Constanza Perez

- **Objetivo:** Medir el conocimiento aprendido de base de datos, web scraping y APIs. Para esto se solicita desarrollar
un sencillo producto de consulta de inmuebles con sus comercios cercano.

### **IMPORTACIÓN DE LIBRERÍAS**

In [1]:
import sqlite3
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException


### **VARIABLES**

In [2]:
api_key = ''
op_contrato = 'arriendo'  # opciones: venta, arriendo, arriendo_temporal
op_inmueble = 'departamentos'  # opciones: dpto, casa, oficina
ubicacion_inmueble = 'las condes'
monto_minimo = 500000
monto_maximo = 600000
cant_paginas = 3
radio_busqueda = '300'
busqueda_rubros = ['restaurante', 'supermercado']

### **WEBSCRAPING**

In [3]:
# Inicializa WebDriver 
driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()))

# Maximiza la ventana
driver.maximize_window()

# Navega al sitio
url = "https://www.portalinmobiliario.com/"
driver.get(url)
time.sleep(3)

#### **Busqueda de Información**

In [4]:
# Interactua con los filtros
try:
    # Filtrar por tipo de contrato
    contrato_boton = driver.find_element(By.XPATH, "//button[@aria-label='Tipo de operación']")
    contrato_boton.click()
    
    # Selecciona el tipo de contrato (Venta, Arriendo, Arriendo Temporal)
    driver.find_element(By.XPATH, f"//span[contains(text(), '{op_contrato.capitalize()}')]").click()

    # Filtra por tipo de inmueble
    inmueble_boton = driver.find_element(By.XPATH, "//button[@aria-label='Tipo de propiedad']")
    inmueble_boton.click()
    
    # Selección del tipo de inmueble (Departamento, Casa, Oficina)
    driver.find_element(By.XPATH, f"//span[contains(text(), '{op_inmueble.capitalize()}')]").click()

    # Filtra por comuna y buscar
    comuna_input = driver.find_element(By.XPATH, "//input[@placeholder='Ingresa comuna o ciudad']")
    comuna_input.send_keys(ubicacion_inmueble)
    time.sleep(2)
    comuna_input.send_keys(Keys.ENTER)

    # Clic en botón buscar con espera de tiempo
    boton_buscar = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//span[@class='andes-button__content' and contains(text(), 'Buscar')]"))
    )
    boton_buscar.click()
    
except Exception as e:
    print("Error durante la interacción", e)

#### **Filtro de Precios**

In [5]:
# Se espera a que cargue la página de resultados
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//input[@data-testid='Minimum-price']"))
)

# Filtro por monto mínimo y monto máximo
minimo_input = driver.find_element(By.XPATH, "//input[@data-testid='Minimum-price']")
maximo_input = driver.find_element(By.XPATH, "//input[@data-testid='Maximum-price']")

minimo_input.clear()
minimo_input.send_keys(str(monto_minimo))

maximo_input.clear()
maximo_input.send_keys(str(monto_maximo))
maximo_input.send_keys(Keys.ENTER)

#### **Función de Scroll**

In [6]:
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

#### **Extracción y Almacenamiento de Resultados**

In [7]:
resultados = []

for pagina in range(cant_paginas):
    scroll_down(driver)

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//li[contains(@class, 'ui-search-layout__item')]"))
    )
    anuncios = driver.find_elements(By.XPATH, "//li[contains(@class, 'ui-search-layout__item')]")

    i = 0
    while i < len(anuncios):
        try:
            anuncios = driver.find_elements(By.XPATH, "//li[contains(@class, 'ui-search-layout__item')]//a")

            if i >= len(anuncios):
                break

            # Hacer clic en el anuncio sin abrir una nueva pestaña
            driver.execute_script("arguments[0].target='_self'; arguments[0].click();", anuncios[i])
            
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'ui-pdp-title'))
            )

            # Extracción de datos
            try:
                titulo = driver.find_element(By.XPATH, "//h1[@class='ui-pdp-title']").text
            except:
                titulo = "No disponible"
            
            precio = driver.find_element(By.XPATH, "//span[@class='andes-money-amount__fraction']").text
            moneda = driver.find_element(By.XPATH, "//span[@class='andes-money-amount__currency-symbol']").text
            link = driver.current_url

            try:
                direccion = driver.find_element(
                    By.XPATH, "//div[@class='ui-vip-location']//p[contains(@class, 'ui-pdp-media__title')]"
                ).text
            except:
                direccion = "No disponible"

            resultados.append([titulo, moneda, precio, direccion, link])

            # Volver a la página anterior sin abrir nueva pestaña
            driver.back()
            
            scroll_down(driver)

            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//li[contains(@class, 'ui-search-layout__item')]"))
            )

            i += 1

        except Exception as e:
            print(f"Error al extraer datos del anuncio: {e}")
            i += 1
            continue

    if pagina < cant_paginas - 1:
        try:
            siguiente_boton = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//li[contains(@class, 'andes-pagination__button--next')]/a"))
            )
            siguiente_boton.click()

            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//li[contains(@class, 'ui-search-layout__item')]"))
            )
        except Exception as e:
            print(f"No se encontró el botón 'Siguiente' o no se pudo hacer clic: {e}")
            break

df_resultados = pd.DataFrame(resultados, columns=['Título', 'Moneda', 'Precio', 'Dirección', 'Enlace'])


#### **Impresión del Dataframe**

In [8]:
pd.set_option('display.max_rows', None)
df_resultados

,Título,Moneda,Precio,Dirección,Enlace
0,Somma Parque Bustamante - Metro Irarrázaval Ñuñoa,$,518.140,"San Eugenio 401, Parque San Eugenio - Metro Ñu...",https://www.portalinmobiliario.com/arriendo/de...
1,IMU San Cristóbal,UF,14,"Inés Matte Urrejola 900, Providencia, Las Cond...",https://www.portalinmobiliario.com/arriendo/de...
2,INSITU Irarrázaval,$,504.824,"Avenida Irarrázaval 2100 - 2400, Metro Ñuñoa, ...",https://www.portalinmobiliario.com/arriendo/de...
3,Activa Juan Mitjans,$,550.000,"Juan Mitjans 135, Metro Rodrigo de Araya, Macu...",https://www.portalinmobiliario.com/arriendo/de...
4,Arriendo Depto a pasos del Metro Chile-España ...,$,500.700,"Av. Irarrázaval 2899, Plaza Ñuñoa, Ñuñoa, RM (...",https://www.portalinmobiliario.com/arriendo/de...
5,Nomad Bellet,UF,13,"Antonio Bellet 126, Providencia, Pedro de Vald...",https://www.portalinmobiliario.com/arriendo/de...
6,Arriendo Departamento Nuevo 2d+2b Avenida Espa...,$,530.000,"Av. España 702, Santiago, Barrio República, Sa...",https://www.portalinmobiliario.com/MLC-1556326...
7,"Arriendo 2d+2b Carmen 121, Metro Santa Lucía",$,500.000,"Carmen 121, Santiago, Santa Isabel, Santiago, ...",https://www.portalinmobiliario.com/MLC-2808039...
8,Departamento En Arriendo De 2 Dorm. En Concepción,$,500.000,"Orompello 1470, Concepción, Barrio Poniente, C...",https://www.portalinmobiliario.com/MLC-2814801...
9,Departamento Walker Martínez Id: 143802,$,530.000,"Walker Martínez, La Florida, RM (Metropolitana)",https://www.portalinmobiliario.com/MLC-1552324...


In [9]:
# Cerrar el WebDriver principal
driver.quit()

#### **Obtención del valor de la UF**

In [10]:
# Nueva sesión de WebDriver para obtener la UF
def obtener_valor_uf():
    driver_uf = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()))
    driver_uf.get("https://valoruf.cl/")
    time.sleep(3)
    valor_uf = WebDriverWait(driver_uf, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "vpr"))
    ).text
    driver_uf.quit()
    valor_uf = float(valor_uf.replace('$', '').replace('.', '').replace(',', '.'))
    return valor_uf

#### **Valor de la UF**

In [11]:
valor_uf = obtener_valor_uf()
print(f"Valor de la UF hoy: {valor_uf} CLP")

Valor de la UF hoy: 38436.51 CLP


#### **Convertir valores en UF a CLP**

In [12]:
for index, row in df_resultados.iterrows():
    if row['Moneda'] == 'UF':
        # Calcular el valor en CLP
        precio_en_clp = float(row['Precio'].replace('.', '')) * valor_uf
        
        # Redondear y convertir a entero
        df_resultados.at[index, 'Precio'] = int(round(precio_en_clp))
        df_resultados.at[index, 'Moneda'] = '$'

In [13]:
df_resultados['Moneda'] = df_resultados['Moneda'].replace({'CLP': '$'})

#### **Normalización de Precios**

In [14]:
df_resultados['Precio'] = df_resultados['Precio'].astype(str).str.replace('.', '').astype(float).astype(int)

#### **Impresión Dataframe**

In [15]:
pd.set_option('display.max_rows', None)
df_resultados

,Título,Moneda,Precio,Dirección,Enlace
0,Somma Parque Bustamante - Metro Irarrázaval Ñuñoa,$,518140,"San Eugenio 401, Parque San Eugenio - Metro Ñu...",https://www.portalinmobiliario.com/arriendo/de...
1,IMU San Cristóbal,$,538111,"Inés Matte Urrejola 900, Providencia, Las Cond...",https://www.portalinmobiliario.com/arriendo/de...
2,INSITU Irarrázaval,$,504824,"Avenida Irarrázaval 2100 - 2400, Metro Ñuñoa, ...",https://www.portalinmobiliario.com/arriendo/de...
3,Activa Juan Mitjans,$,550000,"Juan Mitjans 135, Metro Rodrigo de Araya, Macu...",https://www.portalinmobiliario.com/arriendo/de...
4,Arriendo Depto a pasos del Metro Chile-España ...,$,500700,"Av. Irarrázaval 2899, Plaza Ñuñoa, Ñuñoa, RM (...",https://www.portalinmobiliario.com/arriendo/de...
5,Nomad Bellet,$,499675,"Antonio Bellet 126, Providencia, Pedro de Vald...",https://www.portalinmobiliario.com/arriendo/de...
6,Arriendo Departamento Nuevo 2d+2b Avenida Espa...,$,530000,"Av. España 702, Santiago, Barrio República, Sa...",https://www.portalinmobiliario.com/MLC-1556326...
7,"Arriendo 2d+2b Carmen 121, Metro Santa Lucía",$,500000,"Carmen 121, Santiago, Santa Isabel, Santiago, ...",https://www.portalinmobiliario.com/MLC-2808039...
8,Departamento En Arriendo De 2 Dorm. En Concepción,$,500000,"Orompello 1470, Concepción, Barrio Poniente, C...",https://www.portalinmobiliario.com/MLC-2814801...
9,Departamento Walker Martínez Id: 143802,$,530000,"Walker Martínez, La Florida, RM (Metropolitana)",https://www.portalinmobiliario.com/MLC-1552324...
